In [2]:
# AI Sales Agent - Google Colab Notebook

# **Step 1: Install Required Libraries**
!pip install predictionguard langchain langchain_community pandas python-dotenv cryptography


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [13]:
# **Step 2: Import Required Libraries**
import os
import json
import pandas as pd
from getpass import getpass
from dotenv import load_dotenv
from cryptography.fernet import Fernet
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import PredictionGuard

In [14]:
# **Step 3: Secure API Key Input**
load_dotenv()


False

In [15]:
pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

Enter your Prediction Guard access token: ··········


In [16]:
# **Step 4: Initialize Prediction Guard Model**
llm = PredictionGuard(
    model="Hermes-3-Llama-3.1-70B",
    predictionguard_api_key=os.environ['PREDICTIONGUARD_TOKEN'],
    temperature=0.75,
    max_tokens=100,
    stop=["000"]
)

In [17]:
# **Step 5: Meeting Notes (Example Input for Processing)**
meeting_notes = [
    {
        "date": "2025-02-05",
        "attendees": "University IT Representative, Prediction Guard Team",
        "discussion_points": [
            "The university has significant faculty and student adoption of a 'free' AI-powered chat environment tied to their existing cloud infrastructure.",
            "Concerns were raised about the high cost of enterprise AI solutions, which require per-user fees.",
            "The university is exploring alternative AI solutions that provide similar functionalities while keeping costs manageable."
        ],
        "next_steps": [
            "Prediction Guard to provide further insights on implementation and integration for institutional use.",
            "University Representative to present the proposed solution to senior management and decision-makers."
        ]
    },
    {
        "date": "2025-02-06",
        "attendees": "CTO from a Manufacturing Software Provider, Prediction Guard Team",
        "discussion_points": [
            "The company specializes in Manufacturing Execution Systems (MES) for aerospace and defense.",
            "They seek a custom GPT tailored to their domain for internal and customer support.",
            "A pilot deployment is acceptable, but self-hosting is the preferred long-term solution."
        ],
        "next_steps": [
            "Prediction Guard to send relevant documentation and pilot agreement.",
            "Company CTO to discuss with CFO and CEO and send NDA for deeper discussions."
        ]
    },
    {
        "date": "2025-02-07",
        "attendees": "CIO of a Healthcare Analytics Firm, Prediction Guard Team",
        "discussion_points": [
            "The company aims to enhance predictive modeling for patient risk analysis.",
            "They are exploring AI-driven dashboards for real-time patient health monitoring.",
            "Compliance with HIPAA and other healthcare regulations is a key concern."
        ],
        "next_steps": [
            "Prediction Guard to outline compliance and security measures in AI deployment.",
            "CIO to schedule a follow-up with the legal and compliance team."
        ]
    }
]

In [18]:
# **Step 6: Encrypt and Decrypt Meeting Notes**
def generate_encryption_key():
    key = Fernet.generate_key()
    return key

def encrypt_meeting_notes(notes, key):
    cipher = Fernet(key)
    encrypted_data = cipher.encrypt(json.dumps(notes).encode("utf-8"))
    return encrypted_data

def decrypt_meeting_notes(encrypted_data, key):
    cipher = Fernet(key)
    decrypted_data = cipher.decrypt(encrypted_data).decode("utf-8")
    return json.loads(decrypted_data)

# Generate Encryption Key
enc_key = generate_encryption_key()

# Encrypt Notes
encrypted_notes = encrypt_meeting_notes(meeting_notes, enc_key)

# Decrypt Notes for Processing
decrypted_notes = decrypt_meeting_notes(encrypted_notes, enc_key)
print("Meeting Notes Successfully Decrypted!")

Meeting Notes Successfully Decrypted!


In [19]:
# **Step 7: Meeting Summary and Lead Scoring Using AI and Keyword Analysis**
summary_prompt_template = PromptTemplate(
    input_variables=["discussion"],
    template="""
    Summarize the following meeting discussion points in bullet points:
    {discussion}
    """
)

summary_chain = LLMChain(llm=llm, prompt=summary_prompt_template)

def summarize_meeting(discussion):
    """Generate AI-driven summary of meeting discussion points."""
    try:
        response = summary_chain.run({"discussion": "\n".join(discussion)})
        return response.strip() if response else "Summarization failed."
    except Exception as e:
        return f"Error: {str(e)}"

lead_score_prompt_template = PromptTemplate(
    input_variables=["discussion"],
    template="""
    Based on the following meeting discussion points and what Prediction Guard offers, assess the quality of this sales lead.
    Consider their interest, urgency, and alignment with Prediction Guard's solutions.
    Provide a lead score from 0 to 100 and justify your reasoning:
    {discussion}
    """
)

lead_score_chain = LLMChain(llm=llm, prompt=lead_score_prompt_template)

def assess_lead_quality(discussion):
    """Use LLM to assess the lead quality based on meeting discussion."""
    try:
        response = lead_score_chain.run({"discussion": "\n".join(discussion)})
        return response.strip() if response else "Scoring failed."
    except Exception as e:
        return f"Error: {str(e)}"

def keyword_based_scoring(summary):
    """Secondary keyword-based lead scoring."""
    if "high priority" in summary.lower() or "immediate interest" in summary.lower():
        return "Hot Lead (Score: 90+)"
    elif "follow-up needed" in summary.lower() or "budget discussion" in summary.lower():
        return "Warm Lead (Score: 70-89)"
    else:
        return "Cold Lead (Score: <70)"

In [20]:
# **Step 8: Process Meeting Notes**
processed_data = []

for meeting in decrypted_notes:
    meeting_summary = summarize_meeting(meeting["discussion_points"])
    lead_quality_assessment = assess_lead_quality(meeting["discussion_points"])
    keyword_score = keyword_based_scoring(lead_quality_assessment)

    processed_data.append({
        "Date": meeting["date"],
        "Attendees": meeting["attendees"],
        "Summary": meeting_summary,
        "Next Steps": "\n".join(meeting["next_steps"]),
        "Lead Quality Assessment": lead_quality_assessment,
        "Keyword-Based Score": keyword_score
    })

In [21]:
# **Step 9: Convert to DataFrame and Save**
df = pd.DataFrame(processed_data)
df.to_csv("lead_scoring_results.csv", index=False)

print("Lead Scoring Completed! Here are the results:")
df.head()


Lead Scoring Completed! Here are the results:


,Date,Attendees,Summary,Next Steps,Lead Quality Assessment,Keyword-Based Score
0,2025-02-05,"University IT Representative, Prediction Guard...",- University has high adoption of AI-powered c...,Prediction Guard to provide further insights o...,Meeting discussion points:\n 1. The univers...,Cold Lead (Score: <70)
1,2025-02-06,"CTO from a Manufacturing Software Provider, Pr...",- Company specializes in Manufacturing Executi...,Prediction Guard to send relevant documentatio...,The lead score is 85. The company's interest i...,Cold Lead (Score: <70)
2,2025-02-07,"CIO of a Healthcare Analytics Firm, Prediction...",- Enhance predictive modeling for patient risk...,Prediction Guard to outline compliance and sec...,The company is in the early stages of evaluati...,Cold Lead (Score: <70)
